# Cartographie des Musées de France : Une Exploration Géographique


Le présent notebook explore la répartition géographique des musées en France en utilisant les données ouvertes fournies par le gouvernement français. Les données utilisées proviennent de l'URL suivante : https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/liste-et-localisation-des-musees-de-france/exports/parquet?lang=fr&timezone=Europe%2FBerlin. Ce jeu de données contient des informations détaillées sur les musées de France, notamment leur localisation, leur appellation officielle, et d'autres détails administratifs. L'objectif de cette analyse est de préparer les données pour créer une carte choroplèthe représentant le nombre de musées par département français.


Pour atteindre cet objectif, nous utiliserons les deux premiers caractères du code postal comme code département. Cela nous permettra d'agréger les données au niveau départemental et de visualiser la répartition des musées à travers la France. La carte choroplèthe sera créée en utilisant les bibliothèques Folium et Pandas, offrant ainsi une représentation visuelle claire et interactive de la densité des musées dans chaque département.


## Méthodologie

La méthodologie employée dans ce notebook consiste à importer les données, à les nettoyer et à les agréger au niveau départemental en utilisant le code postal. Nous utiliserons DuckDB pour exécuter une requête SQL qui extrait les deux premiers caractères du code postal et compte le nombre de musées par département. Les résultats seront ensuite utilisés pour créer une carte choroplèthe avec Folium, où chaque département sera coloré en fonction du nombre de musées qu'il contient. Cette approche nous permettra de visualiser efficacement la répartition géographique des musées en France.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [1]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/liste-et-localisation-des-musees-de-france/exports/parquet?lang=fr&timezone=Europe%2FBerlin", safe_mode=True)
print("✅ Données chargées avec succès")

## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [2]:
# Exécution de la requête
df = run_query(""" SELECT 
  SUBSTRING("code_postal", 1, 2) AS code_dept,
  COUNT(*) AS nb_musees
FROM 
  loaded_dataset
GROUP BY 
  SUBSTRING("code_postal", 1, 2)
ORDER BY 
  code_dept """)
print(f"Résultats : {len(df)} lignes")
df.head()

## 📈 Visualisation

La bibliothèque principale utilisée est Folium, qui est une bibliothèque Python de visualisation de données géospatiales. Cette technologie est adaptée pour représenter des données géographiques, telles que les départements français, sur une carte interactive. Folium permet de créer des cartes choropleth avec des informations détaillées au survol ou au clic, ce qui est idéal pour visualiser des données régionales.

In [3]:
import pandas as pd
import duckdb as ddb
import folium
import pandas as pd
import requests
import json

# Récupération du fichier GeoJSON des départements
geojson_url = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
geojson_data = requests.get(geojson_url).json()

# Nettoyage du df pour correspondre aux codes départementaux du GeoJSON
df['code_dept'] = df['code_dept'].astype(str).str.zfill(2)

# Création de la carte centrée sur la France
dataviz = folium.Map(location=[46.9, 2.2], tiles='CartoDB positron', zoom_start=6)

# Ajout de la couche choropleth
folium.Choropleth(
    geo_data=geojson_data,
    name='choropleth',
    data=df,
    columns=['code_dept', 'nb_musees'],
    key_on='feature.properties.code',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white',
    line_weight=1,
    legend_name='Nombre de musées',
    smooth_factor=0,
    bins=[0, 5, 10, 15, 25, 50],
    reset=True,
    highlight=True
).add_to(dataviz)

# Ajout des infobulles détaillées
for feature in geojson_data['features']:
    dept_code = feature['properties']['code']
    
    if dept_code in df['code_dept'].values:
        nb_musees = df[df['code_dept'] == dept_code]['nb_musees'].values[0]
        dept_name = feature['properties']['nom']
        
        folium.Tooltip(
            f"<b>{dept_name} ({dept_code})</b><br>"
            f"Musées : {nb_musees}"
        ).add_child(
            folium.Popup(
                f"<h4>{dept_name} ({dept_code})</h4>"
                f"<strong>Nombre de musées :</strong> {nb_musees}",
                max_width=300
            )
        ).add_to(
            folium.GeoJson(
                feature,
                style_function=lambda x: {
                    'fillOpacity': 0,
                    'color': '#333333',
                    'weight': 2,
                    'fillColor': 'transparent'
                }
            ).add_to(dataviz)
        )

# Ajout du contrôle des couches
folium.LayerControl(collapsed=False).add_to(dataviz)

# Style personnalisé
dataviz.get_root().html.add_child(folium.Element("""
    <style>
        .legend {font-family: 'Helvetica Neue', Arial, sans-serif; font-size: 14px;}
        .info h4 {margin: 0 0 5px; font-size: 16px; color: #333;}
        .folium-tooltip {font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; font-size: 12px;}
    </style>
"""))
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*